In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
#%matplotlib inline
#import seaborn as sns; sns.set()
import numpy as np

from sys import platform as _platform

#if _platform =='linux2':
#    path = '../data/data_sleep/' 
#else:
    #mets ton path ici et ça devrait marcher :)
#    path = "data"
path="data/"
#path = '../data/data_sleep/' 


In [2]:
create_test = True
if(create_test):
    data=pd.read_csv(path+"input_test.csv")
else: 
    data=pd.read_csv(path+"input_train.csv")
    labels=pd.read_csv(path+"challenge_output_data_training_file_sleep_stages_classification.csv", sep=";")

In [5]:
data=data.set_index("ID")

In [6]:
def filt(y,low, high, freq): 
    filter_array = [i for i,f in enumerate(freq) if f>=low and f<=high]
    return np.array([y[i] for i in filter_array])

In [14]:
import re 
def make_eeg_features(train, low, high):
    fs = 250 # in Hz
    N = fs * 15
    train_eeg = train.filter(regex='EEG[0-9]*').values
    X_fft = np.fft.fft(train_eeg)
    freq = np.fft.fftfreq(N, 1./fs)
    filtered_freq = filt(freq,low,high,freq)
    X_fft = np.apply_along_axis(lambda x: filt(x,low,high,freq), 1 , 1./N * abs(X_fft))
    X_fft = pd.DataFrame(X_fft, columns= ["freq"+str(f) for f in filtered_freq], index=train.index)
    return X_fft

In [15]:
#X_fft = make_eeg_features(data, 0.4, 50)
#if(create_test):
#    X_fft.to_csv("data/fft_eeg_test.csv", index=False)
#else :
#    X_fft.to_csv("data/fft_eeg.csv", index=False)


In [16]:
def make_acc_features(train, low, high):
    fs = 10 #in Hz
    N = fs * 15 #number of samples
    accelaration_names = ['ACC_X.','ACC_Y.','ACC_Z.']
    result = []
    for name in accelaration_names:
        train_eeg = train.filter(regex=name+'[0-9]*').values
        X_fft = np.fft.fft(train_eeg)
        freq = np.fft.fftfreq(N, 1./fs)
        filtered_freq = filt(freq,low,high,freq)
        X_fft = np.apply_along_axis(lambda x: filt(x,low,high,freq), 1 , 1./N *abs(X_fft))
        X_fft = pd.DataFrame(X_fft, columns= [name+"freq"+str(f) for f in filtered_freq])
        result.append(X_fft)
    return pd.concat(result,axis = 1)

In [17]:
#fft_acc = make_acc_features(data, 0, 200)
#if(create_test):
#    fft_acc.to_csv("data/fft_acc_test.csv", index=False)
#else:
#    fft_acc.to_csv("data/fft_acc.csv", index=False)


In [18]:
def make_features_fft(create_test =False, make_files = True):
    #si create_test = False: fait les trucs de train, sinon les trucs de test
    #si make_files =True, crée les fichiers dans data
    X_fft = make_eeg_features(data, 0.4, 50)
    if(make_files):
        if(create_test):
            X_fft.to_csv("data/fft_eeg_test.csv", index=True)
        else :
            X_fft.to_csv("data/fft_eeg.csv", index=True)
    
    fft_acc = make_acc_features(data, 0, 200)
    if(make_files):    
        if(create_test):
            fft_acc.to_csv("data/fft_acc_test.csv", index=True)
        else:
            fft_acc.to_csv("data/fft_acc.csv", index=True)
    return X_fft, fft_acc

fft_eeg, fft_acc= make_features_fft(create_test = create_test)

In [19]:
fft_eeg.head()

,freq0.4,freq0.466666666667,freq0.533333333333,freq0.6,freq0.666666666667,freq0.733333333333,freq0.8,freq0.866666666667,freq0.933333333333,freq1.0,...,freq49.4,freq49.4666666667,freq49.5333333333,freq49.6,freq49.6666666667,freq49.7333333333,freq49.8,freq49.8666666667,freq49.9333333333,freq50.0
ID,,,,,,,,,,,,,,,,,,,,,
ID00001,0.369788,0.057286,0.167872,0.541040,0.507652,0.654178,0.195964,0.173139,0.508839,0.398760,...,0.005252,0.008828,0.003025,0.000848,0.001027,0.001472,0.000930,0.001175,0.001289,0.001434
ID00002,2.103583,2.591920,2.237481,0.604488,3.185015,3.908599,4.287624,1.953263,2.170488,2.198010,...,0.009806,0.026986,0.023334,0.001033,0.005210,0.005055,0.002548,0.003436,0.003959,0.004286
ID00003,0.429960,1.027026,0.967975,0.929622,0.535382,0.089322,0.614258,0.248686,0.836322,1.321965,...,0.015784,0.040056,0.019005,0.003932,0.010476,0.006010,0.005508,0.005047,0.005222,0.006066
ID00004,0.474867,1.225553,0.351758,0.186687,1.514145,1.029752,1.490744,0.819839,0.711146,0.927825,...,0.066837,0.013014,0.005082,0.026141,0.008572,0.007097,0.004240,0.005685,0.004562,0.004746
ID00005,6.532762,5.896757,8.322209,7.980773,5.871436,5.003608,2.261613,3.247045,4.272142,2.452262,...,0.023186,0.056814,0.005277,0.004190,0.014972,0.012582,0.009623,0.010150,0.009800,0.009330


In [20]:
def count_by_zero(array):
    """
    Counts the number of times the signal crosses through zero
    """
    counts = 0
    for i in range(0, len(array)-1):
        if np.sign(array[i]) != np.sign(array[i+1]):
            counts+=1
    return counts

In [21]:
def make_signal_dic(data):
    """
    Takes train or test data and creates a dictionary separating each signal
    """
    signals = ["EEG", "ACC_X", "ACC_Y", "ACC_Z"]
    return {signal: data.filter(regex= "^"+signal+".*") 
                for signal in signals
    }

In [22]:
def create_stat_features(signal, signal_name):
    """
    Inputs a signal, outputs some important statistics information about it
    """

    quantiles = signal.quantile(np.linspace(0,1,11), axis=1).T
    quantiles.columns = ["{}_quantile_{}".format(signal_name, q) for q in range(0,110,10)]
    
    moments = pd.concat(
        [signal.mean(axis=1), signal.var(axis=1), signal.skew(axis=1), signal.kurt(axis=1)], 
        axis=1
    )
    moments.columns= [name.format(signal_name) for name in ["mean_{}", "var_{}", "skew_{}", "kurt_{}"]]
    
    return pd.concat([quantiles, moments], axis=1)

In [23]:
signals = make_signal_dic(data)

In [24]:
def make_stats(create_test =False, make_files = True):
    "takes the dict of signals and returns descriptive stats for EEG and ACC"
    stats = {
        signal: create_stat_features(signals[signal], signal)
        for signal in signals.keys()
    }
    stats_EEG = pd.concat(
        [
            stats["EEG"], 
            pd.Series(signals["EEG"].apply(count_by_zero, axis=1), name="EEG_through_0")
        ],
        axis=1
        )
    if make_files:
        if create_test:
            stats_EEG.to_csv("data/stats_eeg_test.csv", index=False)
        else:
            stats_EEG.to_csv("data/stats_eeg.csv", index=False)
    stats_ACC = pd.concat([stats["ACC_{}".format(dim)] for dim in ["X", "Y", "Z"]], axis=1)
    if make_files:
        if create_test:
            stats_ACC.to_csv("data/stats_acc_test.csv", index=False)
        else:
            stats_ACC.to_csv("data/stats_acc.csv", index=False)
    return stats_EEG, stats_ACC

In [ ]:
stats_EEG, stats_ACC=make_stats(create_test =create_test, make_files = True)

In [ ]:
def consolidated_amplitude_accelometer(fft_acc):
    """
    Takes the FFT of the accelerometer for each dimension. Returns the L2 norm of the amplitudes for each sequence
    """
    power_by_dimension = {dim: pd.DataFrame() for dim in ["X", "Y", "Z"]}
    #go through every column
    for col in fft_acc.columns:
        #check which dimension it is related to
        if col[4]=="X":
            #squared of the amplitude for each frequency
            power_by_dimension["X"]["{}".format(col[6:])]=fft_acc[col]**2 
        elif col[4]=="Y":
            power_by_dimension["Y"]["{}".format(col[6:])]=fft_acc[col]**2
        elif col[4]=="Z":
            power_by_dimension["Z"]["{}".format(col[6:])]=fft_acc[col]**2
    return np.sqrt(power_by_dimension["X"] + power_by_dimension["Y"] + power_by_dimension["Z"])

In [ ]:
def speed_accelerometer(dict_accelerometer):
    """
    Takes a dictionary with dataframes for each dimension of the accelerometer. Returns consolidated features 
    (Speed and 1D amplitude)
    """
    #Integral of the acceleration = speed
    return np.sqrt( #sqrt of the
        sum( #sum over
            [
                dict_accelerometer["ACC_{}".format(dim)].sum(axis=1)**2 
                #squared integral of the acceleration
                for dim in ["X", "Y", "Z"] #for each dimension
            ]
        ) 
        
    )

In [ ]:
def make_features_accelerometer(create_test =False, make_files = True):
    ACC_feats = pd.concat(
        [
            consolidated_amplitude_accelometer(fft_acc), 
            pd.Series(speed_accelerometer(signals), name="SPEED")
        ],
        axis=1
        )
    if make_files:
        if create_test:
            ACC_feats.to_csv("data/ACC_feats_test.csv", index=False)
        else:
            ACC_feats.to_csv("data/ACC_feats.csv", index=False)
    return ACC_feats

In [20]:
%pdb

Automatic pdb calling has been turned ON


In [ ]:
ACC_feats=make_features_accelerometer(create_test =create_test)

In [20]:
freq = np.fft.fftfreq(15*10, 1./250)
plt.plot(freq[0:75],result.values[0][0:75])

NameError: name 'plt' is not defined

> <ipython-input-20-75b17ae07f05>(2)<module>()
      1 freq = np.fft.fftfreq(15*10, 1./250)
----> 2 plt.plot(freq[0:75],result.values[0][0:75])

ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 
ipdb> 

KeyboardInterrupt


In [ ]:
X_fft.values[0].shape,filt(np.fft.fftfreq(15*250, 1./250),0.4,50,np.fft.fftfreq(15*250, 1./250)).shape

In [ ]:
freq = np.fft.fftfreq(15*250, 1./250)
plt.plot(filt(freq,0.4,50,freq),np.abs(X_fft.values[31]))
plt.grid()
plt.show()